In [1]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from numba import njit, prange
from scipy import sparse

In [2]:
matplotlib.rcParams["font.size"] = 15

In [3]:
matplotlib.rcParams["font.size"] = 15

In [4]:
matplotlib.rcParams["figure.figsize"] = (10, 10)

In [5]:
sns.set_style("darkgrid", {"axes.facecolor": ".9"})

In [ ]:
def laplacian(N, dx):
    _A_1 = sparse.eye(2*N)
    D_2 = sparse.eye(2*N, k=1) + sparse.eye(2*N, k=-1) - 2 * _A_1
    L = 1 / (dx**2) * (sparse.kron(D_2, _A_1) + sparse.kron(_A_1, D_2))
    return L

In [ ]:
def gradient(p, index):
    return np.gradient(p)[index]